In [1]:
from kalman import *
from animation import Animator
import matplotlib.pyplot as plt
%matplotlib tk

Tento projekt byl vypracován do předmětu modely s neurčitostí. Zadáním projektu bylo použití Kálmánova filtru na nějakém systému. V tomto projektu jsem popisoval pohyb bomby vypuštěné z letadla. Takto vypuštěnou bombu jsem popisoval jako vrh vodorovný. Trajektorií je část paraboly s vrcholem v místě vypuštění.

Dále jsem vytvořil jednoduchý model protiraketového systému, který má bombu detekovat a následně se pokusit ji sestřelit. Bomba je vypštěna ze zakládny, její rychlost je konstantní.

Jak protiraketová střela pozná, kde sestřelit nepřátelskou bombu? Tohoto jsem docílil pomocí Kálmánova filtru. V každém okmžiku predikuji polohu bomby o 200 kroků dopředu. Jakmile se tato predikce dostane na hranici dostřelu protiraketového systému (který jsem stanovil na 5000 m), protiraketový systém vystřelí. Úspěšné sestřelení poznáme tak, že protiraketová střela a bomba jsou od sebe vzdáleny 30 metrů. Protiraketová střela v takové blízkosti exploduje, zničí nepřátelskou bombu a my zaznamenáme úspěšné sestřelení.

Připravil jsem několik modelů, popisující různé situace v našem systému.

Zde definuji konstanty. g je gravitační zrychlení a R je poloměr Země.

In [2]:
R = 6371000
g = 9.81

Nejprve ukážeme model, ve kterém působí pouze gravitační síla.
Ve všech modelech budeme pozorovat $x,y,v_x,v_y$, kde $x$ je poloha v ose x, $y$ je poloha v ose $y$, $v_x$ je x-ová složka rychlosti, $v_y$ je y-ová složka rychlosti. \mathit{\Delta} značí časový krok. Označme $v_x = u$ a $v_y = v$. Platí následující vztahy $$ x_{k+1} = x_{k} + \mathit{\Delta} u_k, \\ y_{k+1} = y_{k} + \mathit{\Delta} v_{k}, \\ u_{k+1} = u_k, \\ v_{k+1} = v_k - g\mathit{\Delta}.$$ Rychlost ve směru osy x je konstantní a ve směru osy y půsoví gravitační síla.

In [3]:
class MissileModel1(ModelProjekt):

    def Ak(self, k, **kwargs):
        return (np.array([[1, 0, self.D, 0],
                          [0, 1, 0, self.D],
                          [0, 0, 1, 0],
                          [0, 0, 0, 1]]))

    def Bk(self, k):
        return (np.array([[0, 0],
                          [0, 0],
                          [self.D, 0],
                          [0, self.D]]))

    def Hk(self, k):
        return (np.array([[1, 0, 0, 0],
                          [0, 1, 0, 0]]))

    def Qk(self, k):
        return 0.01 * np.eye(4)

    def Rk(self, k):
        return 0.001 * np.eye(2)

    def Uk(self, k, **kwargs):
        y = kwargs['y']
        return np.array([0, -g * (1 - (2 * y / R))])

In [4]:
class FriendlyMissile1(ModelProjekt):

    def Ak(self, k, **kwargs):
        return (np.array([[1, 0, self.D, 0],
                          [0, 1, 0, self.D],
                          [0, 0, 1, 0],
                          [0, 0, 0, 1]]))

    def Bk(self, k):
        return (np.array([[0, 0],
                          [0, 0],
                          [self.D, 0],
                          [0, self.D]]))

    def Hk(self, k):
        return (np.array([[1, 0, 0, 0],
                          [0, 1, 0, 0]]))

    def Qk(self, k):
        return 0.01 * np.eye(4)

    def Rk(self, k):
        return 0.001 * np.eye(2)

    def Uk(self, k, **kwargs):
        return np.array([0, 0])


Nastavíme délku časového kroku.

In [5]:
Missile1 = MissileModel1()
Missile1.set_D(0.1)
Friendly = FriendlyMissile1()
Friendly.set_D(0.1)

Vytvoříme instanci třídy Simulation. Ta nám bude simulovat pohyb bomby, protibombového systému a samotnou predikci polohy bomby. Musíme spustit animaci, ve které se nám bude počítat simulace

In [6]:
S = Simulation(enemy_missile=Missile1, friendly_missile=Friendly, mu0=np.array([0,0,0,0]),sigma0=np.diag([1e6, 1e6, 1e6, 1e6]), system_x0=np.array([0, 15000, 555, 0]),  fr_system_x0=np.array([30000, 0, 0, 0]))
anim = Animator(S, 1000)
anim.start_animation()


V následujícm modelu bude na bombu bude působit vítr. Navíc budeme brát v potaz odpor vzduchu ve směru os x a y. Platí Platí následující vztahy $$ x_{k+1} = x_{k} + \mathit{\Delta} u_k, \\ y_{k+1} = y_{k} + \mathit{\Delta} v_{k}, \\ u_{k+1} = u_k - K\mathit{\Delta}u_k + K\mathit{\Delta}w, \\ v_{k+1} = v_k - K\mathit{\Delta}v_k - g\mathit{\Delta}.$$ Rychlost větru w je náhodná díky kovarianční matice $Q_k$. Odpor vzduchu označíme $k_x$ a $k_y$. $m$ značí hmotnost bomby (2 tuny).


In [7]:
wind = 25

class MissileModel2(ModelProjekt):

    kx = 30
    ky = 20
    m = 2000


    def Ak(self, k, **kwargs):
        return (np.array([[1, 0, self.D, 0, 0],
                          [0, 1, 0, self.D, 0],
                          [0, 0, 1 - (self.kx/self.m)*self.D, 0, (self.kx/self.m)*self.D],
                          [0, 0, 0, 1 - (self.ky/self.m)*self.D, 0],
                          [0, 0, 0, 0, 1]]))

    def Bk(self, k):
        return (np.array([[0, 0],
                          [0, 0],
                          [self.D, 0],
                          [0, self.D],
                          [0, 0]]))

    def Hk(self, k):
        return (np.array([[1, 0, 0, 0, 0],
                          [0, 1, 0, 0, 0]]))

    def Qk(self, k):
        Q = 0.01 * np.eye(5)
        Q[4,4] = 0.8
        return Q

    def Rk(self, k):
        return 400 * np.eye(2)

    def Uk(self, k, **kwargs):
        y = kwargs['y']
        return np.array([0, -g * (1 - (2 * y / R))])

In [11]:
Missile2 = MissileModel2()
Missile2.set_D(0.1)
Friendly = FriendlyMissile1()
Friendly.set_D(0.1)

S = Simulation(enemy_missile=Missile2, friendly_missile=Friendly, mu0=np.array([-10000, 20000, 200, 0, 0]),
               sigma0=np.diag([1e2, 1e2, 1e2, 1e2, 1e2]), system_x0=np.array([0, 8000, 555, 0, wind]),
               fr_system_x0=np.array([20000, 0, 0, 0]))
anim = Animator(S, 10000)
anim.start_animation()

In [100]:
plt.figure(figsize=(20, 10))
plt.subplot(221)
plt.plot(S.system.tracex[:, 0], S.system.tracex[:, 1])
plt.xlabel('$x$')
plt.ylabel('$y$')

plt.subplot(223)
plt.plot(S.system.tracet, S.system.tracex[:, 2])
plt.ylabel('$v_x$')
plt.xlabel('$t$')

plt.subplot(224)
plt.plot(S.system.tracet, S.system.tracex[:, 3])
plt.ylabel('$v_y$')
plt.xlabel('$t$')

plt.subplot(222)
plt.plot(S.system.tracet, S.system.tracex[:, 4])
plt.ylabel('$u$')
plt.xlabel('$t$')

Text(0.5, 0, '$t$')

Představme si situaci, kdy chceme zvýšit dolet bomby. Existují tzv. klouzavé bomby, kterým se po vypuštění z bombardéru roztáhnou křídla. Na křídla působí vztlaková síla a odpor vzduchu ve směru osy y je větší. Ukážeme si dva modely a v obou budeme měnit koeficient odporu vzduchu. Křídla bomby jsou ve statické poloze (nijak se nenaklání) a tudíž koeficient odporu se mění v závislosti na úhlu, které svírají mezi sebou složky rychlost vx a vy. Čím větší úhel mezi nimi (to znamená, že bomba klesá strměji), tím menší bude odpor. V každé iteraci vzpočteme koeficient odporu vzduchu.

In [110]:
wind = 2
# koeficient_y = 1
koeficient_y = 20
bomb_x = 0 #  pozice bomby na zacatku v ose x
class MissileModel3(ModelProjekt):

    kx = 10
    ky = 0
    m = 2000


    def Ak(self, k, **kwargs):
        vy = kwargs['vy']
        vx = kwargs['vx']
        self.ky = 50 + (1/(np.arctan(abs(vy+0.001) / abs(vx+0.001))))*koeficient_y

        return (np.array([[1, 0, self.D, 0, 0],
                          [0, 1, 0, self.D, 0],
                          [0, 0, 1 - (self.kx/self.m)*self.D, 0, (self.kx/self.m)*self.D],
                          [0, 0, 0, 1 - (self.ky/self.m)*self.D, 0],
                          [0, 0, 0, 0, 1]]))

    def Bk(self, k):
        return (np.array([[0, 0],
                          [0, 0],
                          [self.D, 0],
                          [0, self.D],
                          [0, 0]]))

    def Hk(self, k):
        return (np.array([[1, 0, 0, 0, 0],
                          [0, 1, 0, 0, 0]]))

    def Qk(self, k):
        Q = 0.01 * np.eye(5)
        Q[4,4] = 0.5
        return Q

    def Rk(self, k):
        return 400 * np.eye(2)

    def Uk(self, k, **kwargs):
        y = kwargs['y']


        return np.array([0, -g * (1 - (2 * y / R))])


In [111]:
Missile3 = MissileModel3()
Missile3.set_D(0.1)
Friendly = FriendlyMissile1()
Friendly.set_D(0.1)

In [112]:
S = Simulation(enemy_missile=Missile3, friendly_missile=Friendly, mu0=np.array([-10000, 20000, 200, 0, 0]),
               sigma0=np.diag([1e6, 1e6, 1e6, 1e6, 1e6]), system_x0=np.array([bomb_x, 8000, 555, 0, wind]),
               fr_system_x0=np.array([26000, 0, 0, 0]))

anim = Animator(S, 10000)
anim.start_animation()

In [113]:
plt.figure(figsize=(20, 10))
plt.subplot(221)
plt.plot(S.system.tracex[:, 0], S.system.tracex[:, 1])
plt.xlabel('$x$')
plt.ylabel('$y$')

plt.subplot(223)
plt.plot(S.system.tracet, S.system.tracex[:, 2])
plt.ylabel('$v_x$')
plt.xlabel('$t$')

plt.subplot(224)
plt.plot(S.system.tracet, S.system.tracex[:, 3])
plt.ylabel('$v_y$')
plt.xlabel('$t$')

plt.subplot(222)
plt.plot(S.system.tracet, S.system.tracex[:, 4])
plt.ylabel('$u$')
plt.xlabel('$t$')

Text(0.5, 0, '$t$')